In [11]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

from sklearn.metrics import roc_auc_score, roc_curve, auc

import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
# import collections

import os
import subprocess
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

from sklearn.metrics import roc_auc_score, roc_curve, auc


def getData(meta, X_raw, tgt, future):
	"""
	Let us say that target 'tgt' is the index of a tile, 
	so we predict for all three variables, CASE
	"""
	
	AR = meta.loc[tgt, 'CASE']
	#PR = meta.loc[tgt, 'BURGLARY-THEFT-MOTOR_VEHICLE_THEFT']
	#AS = meta.loc[tgt, 'HOMICIDE-ASSAULT-BATTERY']
	
	#print('Arrest: {},\nProperty: {},\nAssault: {}.'.format(AR, PR, AS))
	
	X = X_raw.T
	Y = X_raw[[AR]].T

	# get train and test
	# Train is the first two years (2014, 2015), so of length 365 * 2 = 730
	# test is the last year (2016, a leap year), so of length 366
	X_train = X[:30 - future]
	Y_train = Y[future: 30]

	X_test = X[30 - future: -future]
	Y_test = Y[30:]
	
	# Reshape to fit RNN
	# The dimension of RNN input/output is (num_samples, length_in_time, data_dimension)
	X_train = X_train.reshape(1, *X_train.shape)
	Y_train = Y_train.reshape(1, *Y_train.shape)

	X_test = X_test.reshape(1, *X_test.shape)
	Y_test = Y_test.reshape(1, *Y_test.shape)
	
	print('Training data: input dim = {}, output dim = {}'.format(X_train.shape, Y_train.shape))
	print('Out-sample data: input dim = {}, output dim = {}'.format(X_test.shape, Y_test.shape))

	return X_train, Y_train, X_test, Y_test


def train(X_train, Y_train, epochs=200):

	model = tf.keras.Sequential()

	# Don't use unless you are sure test length is the same
	# as the train length, which is not the case for us
	# model.add(tf.keras.Input(X_train.shape[1:]))  

	# Two LSTM layers
	model.add(LSTM(units=30, input_shape=(None, X_train.shape[-1]), return_sequences=True))
	model.add(LSTM(units=10, return_sequences=True))
	
	# One output layers
	model.add(TimeDistributed(Dense(units=1, activation='sigmoid')))
	
	model.compile(loss='mse', optimizer='adam')

	model.summary()

	model.fit(X_train, Y_train, epochs=epochs, batch_size=1, verbose=1)
	
	return model


def train_3(X_train, Y_train, epochs=200):

	model = tf.keras.Sequential()

	# Don't use unless you are sure test length is the same
	# as the train length, which is not the case for us
	# model.add(tf.keras.Input(X_train.shape[1:]))  

	# Two LSTM layers
	model.add(LSTM(units=50, input_shape=(None, X_train.shape[-1]), return_sequences=True))
	model.add(LSTM(units=10, return_sequences=True))
	model.add(LSTM(units=10, return_sequences=True))
	
	# One output layers
	model.add(TimeDistributed(Dense(units=1, activation='sigmoid')))
	
	model.compile(loss='mse', optimizer='adam')

	model.summary()

	model.fit(X_train, Y_train, epochs=epochs, batch_size=1, verbose=1)
	
	return model



In [12]:
#meta = pd.read_csv('meta.csv', index_col=0)
meta = pd.read_csv('meta2.csv', index_col=0)
#X_raw = np.genfromtxt('CRIME-_2014-01-01_2016-12-31.csv')
X_raw = np.genfromtxt('/home/ishanu/Dropbox/ZED/Research/CCTS40500_/notebooks/time_series_covid/data.csv')


In [13]:
meta

,CASE,lat1,lat1.1,lon1,lon2
31.82571#117.2264#CASE,1,31.82571,31.82571,117.2264,117.2264
40.18238#116.4142#CASE,2,40.18238,40.18238,116.4142,116.4142
30.05718#107.874#CASE,3,30.05718,30.05718,107.8740,107.8740
26.07783#117.9895#CASE,4,26.07783,26.07783,117.9895,117.9895
36.0611#103.8343#CASE,5,36.06110,36.06110,103.8343,103.8343
...,...,...,...,...,...
50.5039#4.4699#CASE,70,50.50390,50.50390,4.4699,4.4699
43.0731#-89.4012#CASE,71,43.07310,43.07310,-89.4012,-89.4012
35.4437#129.638#CASE,72,35.44370,35.44370,129.6380,129.6380
32.7157#-117.1611#CASE,73,32.71570,32.71570,-117.1611,-117.1611


In [14]:
cynetTop = meta.index.values

In [15]:
X_raw.shape

(74, 45)

## An example

In [29]:
center = cynetTop[0]
tile = meta.loc[center, ['lat1', 'lat2','lon1', 'lon2']]
    
future = 2
    
X_train, Y_train, X_test, Y_test = getData(meta, X_raw, center, future)

model = train(X_train, Y_train, epochs=20)
prediction = model.predict(X_test)

/home/ishanu/.local/lib/python3.7/site-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


Training data: input dim = (1, 28, 74), output dim = (1, 28, 1)
Out-sample data: input dim = (1, 15, 74), output dim = (1, 15, 1)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, None, 30)          12600     
_________________________________________________________________
lstm_16 (LSTM)               (None, None, 10)          1640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 1)           11        
Total params: 14,251
Trainable params: 14,251
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1/1 [==============================] - 2s 2s/sample - loss: 22212.4551
Epoch 2/20
1/1 [==============================] - 0s 35ms/sample - loss: 22209.2090
Epoch 3/20
1/1 [==============================] - 0s 32ms/sample - loss: 22205.0723


In [30]:
figTitle = '#'.join(map(str, tile.values)) + '_{}'.format(future)
dfName = figTitle + '.rnnres'

In [31]:
Y_test=np.squeeze(Y_test)
prediction=np.squeeze(prediction)

In [32]:
from sklearn.metrics import roc_auc_score
np.corrcoef(Y_test, prediction)

array([[1.        , 0.84280627],
       [0.84280627, 1.        ]])

In [ ]:
Y_test

In [ ]:
prediction